## This code includes the following features:

### Database Management:
- Creates a SQLite database.
- Defines a table specifically for storing PCB design files.
- Offers basic CRUD (Create, Read, Update, Delete) operations to manage the data within the database.

### Issue Tracking and Management:
- Enables users to create issues to track bugs, enhancements, and other tasks.
- Allows for issue categorization (e.g., bug, enhancement).
- Supports prioritization of issues based on importance.
- Facilitates assignment of issues to specific team members.

### Version Control Integration:
- Integrates basic version control features.
- Allows users to check-in and check-out PCB design files, similar to Git functionality.
- Maintains version history to track changes over time, enabling review of file modifications.


1. Setting up the SQLite Database

In [42]:
import sqlite3

def create_connection(db_file):
    """ Create a database connection to the SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Exception as e:
        print(e)
    return conn

def create_table(conn):
    """ Create table for storing PCB design files """
    try:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS pcb_designs (
                file_id INTEGER PRIMARY KEY,
                file_name TEXT NOT NULL,
                version TEXT,
                creation_date TEXT
            )
        """)
        conn.commit()
    except Exception as e:
        print(e)

# Initialize the database
db_file = 'pcb_plm.db'
conn = create_connection(db_file)
create_table(conn)


2. Basic CRUD Operations

In [15]:
def add_pcb_design(conn, pcb_design):
    """ Add a new PCB design file to the database """
    sql = ''' INSERT INTO pcb_designs(file_name, version, creation_date)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, pcb_design)
    conn.commit()
    return cur.lastrowid

def get_pcb_design(conn, file_id):
    """ Query PCB design by id """
    cur = conn.cursor()
    cur.execute("SELECT * FROM pcb_designs WHERE file_id=?", (file_id,))
    return cur.fetchone()

def update_pcb_design(conn, pcb_design):
    """ Update a PCB design file """
    sql = ''' UPDATE pcb_designs
              SET file_name = ?, version = ?, creation_date = ?
              WHERE file_id = ?'''
    cur = conn.cursor()
    cur.execute(sql, pcb_design)
    conn.commit()

def delete_pcb_design(conn, file_id):
    """ Delete a PCB design file by file_id """
    sql = 'DELETE FROM pcb_designs WHERE file_id=?'
    cur = conn.cursor()
    cur.execute(sql, (file_id,))
    conn.commit()


Example Usage
Here's how you can use these functions:

In [16]:
# Add a new PCB design
pcb_design = ('example_pcb_v1', '1.0', '2024-01-10')
file_id = add_pcb_design(conn, pcb_design)

# Get the newly added design
print(get_pcb_design(conn, file_id))

# Update the PCB design
update_pcb_design(conn, ('example_pcb_v2', '2.0', '2024-01-11', file_id))

# Delete the PCB design
# delete_pcb_design(conn, file_id)


(4, 'example_pcb_v1', '1.0', '2024-01-10')


Issue Tracking and Management
Implement a system for logging, tracking, and managing issues or bugs associated with different PCB designs.
Include functionalities like issue creation, categorization (e.g., bug, enhancement), prioritization, and assignment to team members.

To implement an issue tracking and management system for your PCB design PLM, we'll expand upon the initial SQLite setup. The system will include functionalities for issue creation, categorization, prioritization, and assignment to team members. 

Step 1: Define a Table for Issue Tracking

In [20]:
def create_issue_table(conn):
    """ Create table for storing issues related to PCB designs """
    try:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS pcb_issues (
                issue_id INTEGER PRIMARY KEY,
                pcb_file_id INTEGER,
                description TEXT,
                category TEXT,
                priority INTEGER,
                status TEXT,
                assigned_to TEXT,
                FOREIGN KEY (pcb_file_id) REFERENCES pcb_designs (file_id)
            )
        """)
        conn.commit()
    except Exception as e:
        print(e)
        
create_issue_table(conn)

Step 2: Functions for Issue Tracking Operations
Next, we'll add functions to handle operations such as creating, reading, updating, and deleting issues.

In [21]:
def create_issue(conn, issue):
    """ Add a new issue related to a PCB design file """
    sql = ''' INSERT INTO pcb_issues(pcb_file_id, description, category, priority, status, assigned_to)
              VALUES(?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, issue)
    conn.commit()
    return cur.lastrowid

def get_issue(conn, issue_id):
    """ Query issue by id """
    cur = conn.cursor()
    cur.execute("SELECT * FROM pcb_issues WHERE issue_id=?", (issue_id,))
    return cur.fetchone()

def update_issue(conn, issue):
    """ Update an issue """
    sql = ''' UPDATE pcb_issues
              SET pcb_file_id = ?, description = ?, category = ?, priority = ?, status = ?, assigned_to = ?
              WHERE issue_id = ?'''
    cur = conn.cursor()
    cur.execute(sql, issue)
    conn.commit()

def delete_issue(conn, issue_id):
    """ Delete an issue by issue_id """
    sql = 'DELETE FROM pcb_issues WHERE issue_id=?'
    cur = conn.cursor()
    cur.execute(sql, (issue_id,))
    conn.commit()


Step 3: Example Usage
Here's how you can use these functions to manage issues:

In [24]:
# Create an issue
issue = (1, 'Power supply mismatch', 'bug', 2, 'open', 'John Doe')
issue_id = create_issue(conn, issue)

# Get the created issue
print(get_issue(conn, issue_id))

# Update the issue
update_issue(conn, (1, 'Power supply mismatch', 'bug', 1, 'resolved', 'Jane Smith', issue_id))

# Delete the issue
# delete_issue(conn, issue_id)


(2, 1, 'Power supply mismatch', 'bug', 2, 'open', 'John Doe')


**Version Control Integration**

**Step 1: Extend the PCB Design Table for Version Control** <br>
modify the pcb_designs table to include additional fields necessary for version control, like last_modified and is_checked_out

In [37]:
def modify_pcb_design_table_for_version_control(conn):
    """ Modify the PCB design table to include version control fields """
    try:
        cursor = conn.cursor()
        cursor.execute("""
            ALTER TABLE pcb_designs
            ADD COLUMN last_modified TEXT;
        """)
        cursor.execute("""
            ALTER TABLE pcb_designs
            ADD COLUMN is_checked_out BOOLEAN NOT NULL DEFAULT 0;
        """)
        conn.commit()
    except Exception as e:
        print(e)
        
modify_pcb_design_table_for_version_control(conn)

duplicate column name: last_modified


**Step 2: Functions for Version Control Operations** <br>
Implement functions to handle check-in, check-out, and version history tracking of PCB design files.

In [39]:
def check_out_pcb_design(conn, file_id, user):
    """ Check out a PCB design file for editing """
    sql = ''' UPDATE pcb_designs
              SET is_checked_out = 1,
                  last_modified = datetime('now')
              WHERE file_id = ?'''
    cur = conn.cursor()
    cur.execute(sql, (file_id,))
    conn.commit()
    # Log this action
    log_version_history(conn, file_id, f"Checked out by {user}")

def check_in_pcb_design(conn, file_id, user):
    """ Check in a PCB design file after editing """
    sql = ''' UPDATE pcb_designs
              SET is_checked_out = 0,
                  last_modified = datetime('now')
              WHERE file_id = ?'''
    cur = conn.cursor()
    cur.execute(sql, (file_id,))
    conn.commit()
    # Log this action
    log_version_history(conn, file_id, f"Checked in by {user}")

def log_version_history(conn, file_id, action):
    """ Log the version history of PCB design files """
    sql = ''' INSERT INTO pcb_version_history(file_id, action, timestamp)
              VALUES(?,?,datetime('now')) '''
    cur = conn.cursor()
    cur.execute(sql, (file_id, action))
    conn.commit()


**Step 3: Create a Table for Version History Logs** <br>
A new table is needed to track the version history of each PCB design file.

In [43]:
def create_version_history_table(conn):
    """ Create table for storing version history of PCB designs """
    try:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS pcb_version_history (
                history_id INTEGER PRIMARY KEY,
                file_id INTEGER,
                action TEXT,
                timestamp TEXT,
                FOREIGN KEY (file_id) REFERENCES pcb_designs (file_id)
            )
        """)
        conn.commit()
    except Exception as e:
        print(e)

create_version_history_table(conn)


The log_version_history function is called within the check-in and check-out functions to automatically log these actions. You can expand this to include more detailed logging.

Step 4: Example Usage
Here's how you can use these functions:

In [45]:
# Check out a PCB design
check_out_pcb_design(conn, file_id=1, user='Alice')

# Check in the PCB design
check_in_pcb_design(conn, file_id=1, user='Alice')
